### CNN Training in Frequency Domain

Define basic data parameters:
- **QTD_OF_SUMMARIES**: Represent the number of files wich will be used on training
- **QTD_OF_TEST_SUMMARIES**: Represents the number of files wich will be used on test
- **NR_EPOCHS**: The number of epochs on training
- **BATCH_SIZE**: The batch size used on training

In [ ]:
QTD_OF_SUMMARIES = 15
QTD_OF_TEST_SUMMARIES = 2
NR_EPOCHS = 100
BATCH_SIZE = 200

In [ ]:
import datautils.loader.data_loader as loader
summaries = loader.load_anomalous_summaries()

In [ ]:
import datautils.loader.data_loader as loader
loader.load_mne_data(summaries[:QTD_OF_SUMMARIES])
loader.load_freq_segmented_data(summaries[:QTD_OF_SUMMARIES])

In [ ]:
import datautils.splitter.frequency_splitter as splitter
X_train, X_val, y_train, y_val = splitter.segmented_frequency_data_splitter(summaries[:QTD_OF_SUMMARIES])

In [ ]:
import datautils.balancer.data_balancer as balancer
X_train, y_train = balancer.balance(X_train, y_train)
X_val, y_val = balancer.balance(X_val, y_val)

In [ ]:
X_train[0].shape

In [ ]:
import ia.model.cnn as ia
frequency_cnn = ia.FrequencyCNN(input_shape=(X_train[0].shape))

In [ ]:
frequency_cnn.compile()

In [ ]:
history = frequency_cnn.fit(X_train, y_train, num_epochs=NR_EPOCHS, batch_size=BATCH_SIZE, val_data=X_val, val_labels=y_val)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
frequency_cnn.summary()

In [ ]:
import datautils.loader.data_loader as loader
loader.load_mne_data(summaries[QTD_OF_SUMMARIES + 1 : QTD_OF_SUMMARIES + 1 + QTD_OF_TEST_SUMMARIES])
loader.load_freq_segmented_data(summaries[QTD_OF_SUMMARIES + 1 : QTD_OF_SUMMARIES + 1 + QTD_OF_TEST_SUMMARIES])

In [ ]:
import datautils.splitter.frequency_splitter as splitter
X_test, _, y_test, _ = splitter.segmented_frequency_data_splitter(summaries[QTD_OF_SUMMARIES + 1 : QTD_OF_SUMMARIES + 1 + QTD_OF_TEST_SUMMARIES])

In [ ]:
import numpy as np
predictions = frequency_cnn.model.predict(X_test)
predictions = np.array([prediction > 0.5 for prediction in predictions])

for i in range(len(y_test)):
    print("PREDICTION: " + str(predictions[i] > 0.5) + " | " +  "REAL: " + str(y_test[i]))

In [ ]:
test_loss, test_acc = frequency_cnn.model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')
print(f'Test loss: {test_loss}')

In [ ]:
import ia.metrics.metrics_generator as met
met.Metrics(y_test, predictions).all_metrics()

In [ ]:
import sklearn.metrics as metrics
import matplotlib.pyplot as plt

fpr, tpr, threshold = metrics.roc_curve(y_test, predictions)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()